In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 47.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=afda01d2bea6d6cd1c41dd28d9e5cc90c3580e9cc561bc7e1cfc467abb8b4407
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


# Dependencies

In [2]:
# To create the file's path
import os 
# To import the file into the colab enviroment 
from google.colab import drive
# To connect to Spark and Create Row
from pyspark.sql import SparkSession, Row
# User defined function (UDF) to apply custom functions to dataset, col to return columns
from pyspark.sql.functions import *
# Import data types for casting/converting
from pyspark.sql.types import *
# Import StringIndexer to perform categorical transformation to numerical
from pyspark.ml.feature import StringIndexer
# Transforms multiple rows into a row vector
from pyspark.ml.feature import VectorAssembler
# Linear regression algorithm
from pyspark.ml.regression import LinearRegression
# For model loading
from pyspark.ml.regression import LinearRegressionModel

# Setting Up Drive and Spark Session

In [3]:
# Importing the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Creating the path
path = os.path.join('drive','MyDrive','Colab Datasets','sales_detail_report.csv')
path

'drive/MyDrive/Colab Datasets/sales_detail_report.csv'

In [5]:
# Creating a Spark Session
spark = SparkSession.builder.appName('Spark Project').getOrCreate()
spark

# Data Preprocessing

In [6]:
# Reading the .csv from the mounted drive
df_original = spark.read.csv(path, header=True, sep=';', inferSchema=True)
df = df_original

In [7]:
# Data frame lenght
df.count()

11793728

In [8]:
# First rows
df.head(5)

[Row(Store No.='CL104', POS Terminal No.='CL10401', Transaction No.=180560, Date='01/02/18', Shifted Date=None, VAT %=19, Area HO Currency Code='CLP', Net Amount='27,723.00', VAT Amount='5,267.00', Item No.='BY1610', Size Description='8'),
 Row(Store No.='CL104', POS Terminal No.='CL10401', Transaction No.=180563, Date='01/02/18', Shifted Date=None, VAT %=19, Area HO Currency Code='CLP', Net Amount='-52,933.00', VAT Amount='-10,057.00', Item No.='BA8899', Size Description='9-'),
 Row(Store No.='CL104', POS Terminal No.='CL10401', Transaction No.=180564, Date='01/02/18', Shifted Date=None, VAT %=19, Area HO Currency Code='CLP', Net Amount='52,933.00', VAT Amount='10,057.00', Item No.='BA8899', Size Description='8-'),
 Row(Store No.='CL104', POS Terminal No.='CL10401', Transaction No.=180569, Date='01/02/18', Shifted Date=None, VAT %=19, Area HO Currency Code='CLP', Net Amount='13,437.00', VAT Amount='2,553.00', Item No.='BQ0759', Size Description='XL'),
 Row(Store No.='CL104', POS Termi

In [9]:
# Showing columns name
df.columns

['Store No.',
 'POS Terminal No.',
 'Transaction No.',
 'Date',
 'Shifted Date',
 'VAT %',
 'Area HO Currency Code',
 'Net Amount',
 'VAT Amount',
 'Item No.',
 'Size Description']

In [10]:
# Columns and data types 
df.dtypes

[('Store No.', 'string'),
 ('POS Terminal No.', 'string'),
 ('Transaction No.', 'int'),
 ('Date', 'string'),
 ('Shifted Date', 'string'),
 ('VAT %', 'int'),
 ('Area HO Currency Code', 'string'),
 ('Net Amount', 'string'),
 ('VAT Amount', 'string'),
 ('Item No.', 'string'),
 ('Size Description', 'string')]

In [11]:
# Dropping useless columns 
df = df.drop('POS Terminal No.', 'Shifted Date', 'VAT %', 'Area HO Currency Code', 'VAT Amount', 'Size Description')
df.head(3)

[Row(Store No.='CL104', Transaction No.=180560, Date='01/02/18', Net Amount='27,723.00', Item No.='BY1610'),
 Row(Store No.='CL104', Transaction No.=180563, Date='01/02/18', Net Amount='-52,933.00', Item No.='BA8899'),
 Row(Store No.='CL104', Transaction No.=180564, Date='01/02/18', Net Amount='52,933.00', Item No.='BA8899')]

In [12]:
# Loop through each column and rename it
for col in df.columns:
  new_col = col.replace(" ","_").lower().replace(".","")
  print (f'"{col}" RENAMED TO: "{new_col}"\n')
  df = df.withColumnRenamed(col, new_col)

print (f"{df.columns}")

"Store No." RENAMED TO: "store_no"

"Transaction No." RENAMED TO: "transaction_no"

"Date" RENAMED TO: "date"

"Net Amount" RENAMED TO: "net_amount"

"Item No." RENAMED TO: "item_no"

['store_no', 'transaction_no', 'date', 'net_amount', 'item_no']


In [13]:
# Dropping NA values

In [14]:
df = df.na.drop(how='any')

In [15]:
# Change data type

In [16]:
df.show(2)

+--------+--------------+--------+----------+-------+
|store_no|transaction_no|    date|net_amount|item_no|
+--------+--------------+--------+----------+-------+
|   CL104|        180560|01/02/18| 27,723.00| BY1610|
|   CL104|        180563|01/02/18|-52,933.00| BA8899|
+--------+--------------+--------+----------+-------+
only showing top 2 rows



In [17]:
df.dtypes

[('store_no', 'string'),
 ('transaction_no', 'int'),
 ('date', 'string'),
 ('net_amount', 'string'),
 ('item_no', 'string')]

* 'date' from string to date format
* 'net_amount' from string to int, replace ","

In [18]:
# Apply an user defined function to clean

# Define the UDF
udf_replace = udf(lambda x: x.replace(',',''))

# Select the column to replace then apply the function and cast to change it's data type
df = df.withColumn(
    'net_amount',
    udf_replace(
        df['net_amount']
        ).cast(IntegerType())
    )

df.show(3)
df.dtypes

+--------+--------------+--------+----------+-------+
|store_no|transaction_no|    date|net_amount|item_no|
+--------+--------------+--------+----------+-------+
|   CL104|        180560|01/02/18|     27723| BY1610|
|   CL104|        180563|01/02/18|    -52933| BA8899|
|   CL104|        180564|01/02/18|     52933| BA8899|
+--------+--------------+--------+----------+-------+
only showing top 3 rows



[('store_no', 'string'),
 ('transaction_no', 'int'),
 ('date', 'string'),
 ('net_amount', 'int'),
 ('item_no', 'string')]

In [19]:
# Test: change date string MM/dd/yy to date format yy-MM-dd
df.select('date', to_date('date', 'MM/dd/yy').alias('date_formatted')).show(2)
df.select('date', to_date('date', 'MM/dd/yy').alias('date_formatted')).dtypes

+--------+--------------+
|    date|date_formatted|
+--------+--------------+
|01/02/18|    2018-01-02|
|01/02/18|    2018-01-02|
+--------+--------------+
only showing top 2 rows



[('date', 'string'), ('date_formatted', 'date')]

In [20]:
# Change in place
df = df.withColumn('date', to_date('date', 'MM/dd/yy'))
df.show(3)

+--------+--------------+----------+----------+-------+
|store_no|transaction_no|      date|net_amount|item_no|
+--------+--------------+----------+----------+-------+
|   CL104|        180560|2018-01-02|     27723| BY1610|
|   CL104|        180563|2018-01-02|    -52933| BA8899|
|   CL104|        180564|2018-01-02|     52933| BA8899|
+--------+--------------+----------+----------+-------+
only showing top 3 rows



In [54]:
# Export cleaned data frame into a single partition 
df.repartition(1).write.csv("pyspark_sales_forescasting")

# Modelling

## Vectorizing the Data 

In [21]:
input_cols = ['store_no', 'item_no']
output_cols = [col+'_index' for col in input_cols]

In [22]:
# Create indexer instance
indexer = StringIndexer(
    inputCols=input_cols,
    outputCols=output_cols
    )

In [23]:
# Get a crossed list of non-index and indexed columns
crossed_cols = [col_name for sublist in zip(input_cols, output_cols) for col_name in sublist]
crossed_cols

['store_no', 'store_no_index', 'item_no', 'item_no_index']

In [24]:
# Insert the data frame and then transform it
indexed_df = indexer.fit(df).transform(df)

In [25]:
# Show the new data frame
indexed_df.show(3, truncate=False)

+--------+--------------+----------+----------+-------+--------------+-------------+
|store_no|transaction_no|date      |net_amount|item_no|store_no_index|item_no_index|
+--------+--------------+----------+----------+-------+--------------+-------------+
|CL104   |180560        |2018-01-02|27723     |BY1610 |32.0          |20355.0      |
|CL104   |180563        |2018-01-02|-52933    |BA8899 |32.0          |23292.0      |
|CL104   |180564        |2018-01-02|52933     |BA8899 |32.0          |23292.0      |
+--------+--------------+----------+----------+-------+--------------+-------------+
only showing top 3 rows



In [26]:
# Confirms unique values between non-indexed and indexed columns
for col in crossed_cols:
  print(f"Column '{col}' - Unique Values: {indexed_df.select(col).distinct().count()}")

Column 'store_no' - Unique Values: 78
Column 'store_no_index' - Unique Values: 78
Column 'item_no' - Unique Values: 38452
Column 'item_no_index' - Unique Values: 38452


In [27]:
# Convert date to numeric
indexed_df = indexed_df.withColumn('unix_date', unix_timestamp(indexed_df['date']))

In [28]:
indexed_df.show(3)

+--------+--------------+----------+----------+-------+--------------+-------------+----------+
|store_no|transaction_no|      date|net_amount|item_no|store_no_index|item_no_index| unix_date|
+--------+--------------+----------+----------+-------+--------------+-------------+----------+
|   CL104|        180560|2018-01-02|     27723| BY1610|          32.0|      20355.0|1514851200|
|   CL104|        180563|2018-01-02|    -52933| BA8899|          32.0|      23292.0|1514851200|
|   CL104|        180564|2018-01-02|     52933| BA8899|          32.0|      23292.0|1514851200|
+--------+--------------+----------+----------+-------+--------------+-------------+----------+
only showing top 3 rows



In [29]:
# Selecting the independent variables and dependant variable
x_cols = ['unix_date', 'store_no_index', 'item_no_index']
y_col = 'net_amount'

In [30]:
# Vectorizing the independent variables
featureassembler=VectorAssembler(inputCols=x_cols,

                                outputCol=str(x_cols))

vectorized_df=featureassembler.transform(indexed_df)

In [31]:
vectorized_df.show(3, truncate=False)

+--------+--------------+----------+----------+-------+--------------+-------------+----------+------------------------------------------------+
|store_no|transaction_no|date      |net_amount|item_no|store_no_index|item_no_index|unix_date |['unix_date', 'store_no_index', 'item_no_index']|
+--------+--------------+----------+----------+-------+--------------+-------------+----------+------------------------------------------------+
|CL104   |180560        |2018-01-02|27723     |BY1610 |32.0          |20355.0      |1514851200|[1.5148512E9,32.0,20355.0]                      |
|CL104   |180563        |2018-01-02|-52933    |BA8899 |32.0          |23292.0      |1514851200|[1.5148512E9,32.0,23292.0]                      |
|CL104   |180564        |2018-01-02|52933     |BA8899 |32.0          |23292.0      |1514851200|[1.5148512E9,32.0,23292.0]                      |
+--------+--------------+----------+----------+-------+--------------+-------------+----------+-----------------------------------

In [32]:
# Shriking the data frame with vectorized independent variables and dependent variable
vectorized_df = vectorized_df.select(str(x_cols), y_col)

In [33]:
vectorized_df.show(3, truncate=False)

+------------------------------------------------+----------+
|['unix_date', 'store_no_index', 'item_no_index']|net_amount|
+------------------------------------------------+----------+
|[1.5148512E9,32.0,20355.0]                      |27723     |
|[1.5148512E9,32.0,23292.0]                      |-52933    |
|[1.5148512E9,32.0,23292.0]                      |52933     |
+------------------------------------------------+----------+
only showing top 3 rows



## Fitting the Model

In [34]:
# Split the data into train and test validation
train_data, test_data= vectorized_df.randomSplit([0.8,0.2])

In [35]:
# Creating the Linear Regressor object
regressor = LinearRegression(featuresCol=str(x_cols), labelCol=y_col)


In [36]:
# Fitting the data into
regressor = regressor.fit(train_data)
regressor

LinearRegressionModel: uid=LinearRegression_1f88c8b2e323, numFeatures=3

## Evaluating the Model

In [37]:
# Metrics: Intercept
regressor.intercept

-27602.144433280344

In [38]:
# Metrics: Coefficients
regressor.coefficients

DenseVector([0.0, 52.3357, 0.2306])

In [39]:
# Making predictions about the test data
predicted_results = regressor.evaluate(test_data)

In [40]:
# Showing 
predicted_results.predictions.show(3, truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+------------------------------------------------+----------+------------------+
|['unix_date', 'store_no_index', 'item_no_index']|net_amount|prediction        |
+------------------------------------------------+----------+------------------+
|[1.5148512E9,0.0,2.0]                           |10068     |12625.089021775348|
|[1.5148512E9,0.0,7.0]                           |5034      |12626.24198227236 |
|[1.5148512E9,0.0,7.0]                           |5034      |12626.24198227236 |
+------------------------------------------------+----------+------------------+
only showing top 3 rows



In [41]:
# Performance Metrics
predicted_results.r2, predicted_results.meanAbsoluteError, predicted_results.meanSquaredError

(0.010587234115849942, 12425.96786245647, 412688343.7109269)

## Saving the Model

In [42]:
# Saving the model
regressor.save('pyspark_regressor_sales_forecasting')